In [3]:
options(scipen = 999, repr.plot.width=4, repr.plot.height= 4, warn = -1)

In [5]:
list.of.packages <- c('ggplot2', 'VIM', 'mice', 'mlbench', 'caret', 'caretEnsemble', 'gbm', 'ipred')

new.packages <- list.of.packages[!(list.of.packages %in% installed.packages()[,"Package"])]
if(length(new.packages)) install.packages(new.packages, repos = "https://cran.r-project.org")

library(ggplot2)
library(VIM)
library(mice)
library(mlbench)
library(caret)
library(caretEnsemble)
library(gbm)
library(ipred)

Loading required package: colorspace
Loading required package: grid
Loading required package: data.table
VIM is ready to use. 
 Since version 4.0.0 the GUI is in its own package VIMGUI.

          Please use the package to use the new (and old) GUI.

Suggestions and bug-reports can be submitted at: https://github.com/alexkowa/VIM/issues

Attaching package: 'VIM'

The following object is masked from 'package:datasets':

    sleep

Loading required package: Rcpp
mice 2.25 2015-11-09
Loading required package: lattice

Attaching package: 'caretEnsemble'

The following object is masked from 'package:ggplot2':

    autoplot

Loading required package: survival

Attaching package: 'survival'

The following object is masked from 'package:caret':

    cluster

Loading required package: splines
Loading required package: parallel
Loaded gbm 2.1.1


In [17]:
df_train <- read.csv("train.csv", na.strings = c("", "NA"))
df_test <- read.csv("test.csv", na.strings = c("", "NA"))

id_pass <- df_test$PassengerId
n <- nrow(df_train)

df_test$Survived <- NA
df_train <- rbind(df_train, df_test)

df_train$Survived <- make.names(df_train$Survived)
df_train$Survived[df_train$Survived == "NA."] <- NA
df_train$Survived <- as.factor(df_train$Survived)

df_train$Pclass <- make.names(df_train$Pclass)
df_train$Pclass[df_train$Pclass == "NA."] <- NA
df_train$Pclass <- as.factor(df_train$Pclass)

df_train$Title <- gsub(".*,\\s([a-zA-Z]+)..*", "\\1", df_train$Name)
idx <- which(df_train$Title == "the")
df_train$Title[idx] <- 'Countess'
df_train$Title <- make.names(df_train$Title)
df_train$Title[df_train$Title == "NA."] <- NA
df_train$Title <- as.factor(df_train$Title)

df_train$Sex <- make.names(df_train$Sex)
df_train$Sex[df_train$Sex == "NA."] <- NA
df_train$Sex <- as.factor(df_train$Sex)

df_train$AgeD <- "Bebé"
df_train$AgeD[df_train$Age >= 3] <- "Niño"
df_train$AgeD[df_train$Age >= 18] <- "Adulto"
df_train$AgeD[df_train$Age >= 50] <- "Anciano"
ages <- c("Bebé", "Niño", "Adulto", "Anciano")
df_train$AgeD <- make.names(df_train$AgeD)
df_train$AgeD[df_train$AgeD == "NA."] <- NA
df_train$AgeD <- factor(df_train$AgeD, labels = ages, levels = ages)

df_train$FamSize <- df_train$SibSp + df_train$Parch + 1
df_train$FamSize <- make.names(df_train$FamSize)
df_train$FamSize[df_train$FamSize == "NA."] <- NA
df_train$FamSize <- as.factor(df_train$FamSize)

df_train$SibSp <- make.names(df_train$SibSp)
df_train$SibSp[df_train$SibSp == "NA."] <- NA
df_train$SibSp <- as.factor(df_train$SibSp)

df_train$Parch <- make.names(df_train$Parch)
df_train$Parch[df_train$Parch == "NA."] <- NA
df_train$Parch <- as.factor(df_train$Parch)

df_train$Fare[which(df_train$Fare == 0)] <- NA

df_train$Deck <- substring(df_train$Cabin, 1, 1)
df_train$Deck <- make.names(df_train$Deck)
df_train$Deck[df_train$Deck == "NA."] <- NA
df_train$Deck <- as.factor(df_train$Deck)

df_train$Embarked <- as.character(df_train$Embarked)
df_train$Embarked <- make.names(df_train$Embarked)
df_train$Embarked[df_train$Embarked == "NA."] <- NA
df_train$Embarked <- as.factor(df_train$Embarked)


df_train[, c('PassengerId', 'Ticket', 'Name', 'Age', 'Cabin')] <- NULL

In [18]:
df_temp <- mice(df_train, m=1, maxit=50, meth='pmm', seed=500)
dataset <- complete(df_temp,1)


 iter imp variable
  1   1  Survived  Fare  Embarked  Deck
  2   1  Survived  Fare  Embarked  Deck
  3   1  Survived  Fare  Embarked  Deck
  4   1  Survived  Fare  Embarked  Deck
  5   1  Survived  Fare  Embarked  Deck
  6   1  Survived  Fare  Embarked  Deck
  7   1  Survived  Fare  Embarked  Deck
  8   1  Survived  Fare  Embarked  Deck
  9   1  Survived  Fare  Embarked  Deck
  10   1  Survived  Fare  Embarked  Deck
  11   1  Survived  Fare  Embarked  Deck
  12   1  Survived  Fare  Embarked  Deck
  13   1  Survived  Fare  Embarked  Deck
  14   1  Survived  Fare  Embarked  Deck
  15   1  Survived  Fare  Embarked  Deck
  16   1  Survived  Fare  Embarked  Deck
  17   1  Survived  Fare  Embarked  Deck
  18   1  Survived  Fare  Embarked  Deck
  19   1  Survived  Fare  Embarked  Deck
  20   1  Survived  Fare  Embarked  Deck
  21   1  Survived  Fare  Embarked  Deck
  22   1  Survived  Fare  Embarked  Deck
  23   1  Survived  Fare  Embarked  Deck
  24   1  Survived  Fare  Embarked  Deck
  25 

In [20]:
control <- trainControl(method="repeatedcv", number=10, repeats=3)
seed <- 7
metric <- "Accuracy"
# C5.0
set.seed(seed)
fit.c50 <- train(Survived~., data=dataset, method="C5.0", metric=metric, trControl=control)

In [21]:
df_test <- dataset[(n+1):nrow(dataset),]
df_test$Survived <- NULL

In [45]:
predictions <- predict(fit.c50, df_test)

predictions <- as.character(predictions)
predictions <- substring(predictions, 2)

In [48]:
submit <- data.frame(PassengerId = id_pass, Survived = predictions)
write.csv(submit, file = "firstsubmit.csv", row.names = FALSE, quote = FALSE)

In [44]:
str(id_pass)
str(predictions)

 int [1:418] 892 893 894 895 896 897 898 899 900 901 ...
 int [1:418] 0 0 0 0 1 0 1 0 0 0 ...
